In [1]:
import os
from glob import glob

import numpy as np
import pandas as pd

In [3]:
large_file = glob('/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/*.csv')
large_file

['/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/ratings.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/tags.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/genome-tags.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/genome-scores.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/links.csv',
 '/scratch/work/courses/DSGA1004-2021/movielens/ml-latest/movies.csv']

In [4]:
large_ratings = pd.read_csv(large_file[0])

In [5]:
large_ratings_2 = large_ratings.groupby('userId').filter(lambda x: len(x) > 10)
large_ratings_3 = large_ratings_2.groupby('movieId').filter(lambda x: len(x) > 10)
large_ratings_3

,userId,movieId,rating,timestamp
0,1,307,3.5,1256677221
1,1,481,3.5,1256677456
2,1,1091,1.5,1256677471
3,1,1257,4.5,1256677460
4,1,1449,4.5,1256677264
...,...,...,...,...
27753439,283228,8542,4.5,1379882795
27753440,283228,8712,4.5,1379882751
27753441,283228,34405,4.5,1379882889
27753442,283228,44761,4.5,1354159524


In [6]:
large_ratings_3['rating'].sum()

96740025.0

In [7]:
df = large_ratings_3.sort_values(by='timestamp')
df

,userId,movieId,rating,timestamp
5510411,56769,1176,4.0,789652004
23237827,237556,21,3.0,789652009
23237833,237556,47,5.0,789652009
23237876,237556,1079,3.0,789652009
14712721,150485,1,4.0,822873600
...,...,...,...,...
27486116,280481,2294,2.0,1537945121
27486118,280481,2355,3.0,1537945123
27486035,280481,494,3.0,1537945127
8060104,82922,53519,4.0,1537945130


In [8]:
train, val, test = np.split(df.sample(frac=1), [int(0.6*len(df)), int(0.8*len(df))])

In [9]:
val_sub1, val_sub2 = np.array_split(val, 2)
test_sub1, test_sub2 = np.array_split(test, 2)

In [10]:
train = train.append(val_sub1)
train = train.append(test_sub1)

In [11]:
val = val_sub2
test = test_sub2

In [12]:
# train.to_csv('/scratch/xg693/train_small.txt', index=False, header=False, sep=' ')
# val.to_csv('/scratch/xg693/val_small.txt', index=False, header=False, sep=' ')
# test.to_csv('/scratch/xg693/test_small.txt', index=False, header=False, sep=' ')

In [13]:
train

,userId,movieId,rating,timestamp
3694107,37910,1231,4.0,1377629409
5733652,59067,914,5.0,1017530125
15418825,157601,1293,4.0,1226359822
11118879,114166,1673,4.0,1469038910
27013313,275771,2959,3.5,1448130923
...,...,...,...,...
243835,2395,1208,5.0,899414693
10803850,111114,4886,4.0,1392690415
11706717,120070,2336,4.5,1287016997
19570162,199657,1,4.0,860796913


In [14]:
train_rating_sum = train.groupby('movieId')['rating'].sum() 
train_rating = train_rating_sum / train.groupby('movieId').size()

In [15]:
import operator

sorted_train = dict(sorted(train_rating.items(), key=operator.itemgetter(1), reverse=True))

In [16]:
movie_100 = list(sorted_train.keys())[: 100]

In [17]:
val_new = val[val['movieId'].isin(movie_100)]
test_new = test[test['movieId'].isin(movie_100)]

In [18]:
def MAP_rmse(df):
    cnt = 0
    re = 0

    for i in df['userId'].unique():
        table = df[df['userId']==i]
        temp = table.movieId.to_list()

        for j in temp:
            rating = table[table['movieId']==j].rating.to_list()[0]
            re = re + (rating - sorted_train.get(j))   
            cnt = cnt + 1
            
    print("MAP score: ", str(re / cnt), "\nRMSE score: ", str(re ** 2 / cnt))

In [19]:
# Evaluation for validation 
MAP_rmse(val_new)

MAP score:  -0.0014647713069120276 
RMSE score:  0.28276483657382107


In [20]:
# Evaluation for test
MAP_rmse(test_new)

MAP score:  -0.0021253892143432537 
RMSE score:  0.5955490699943393
